<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/03_embeddings_sherlock_holmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings de Palabras a partir de libros de Sherlock Holmes


# 1) Carga de Datos

Carga libros publicados en Project Gutenberg y elimina el header/footer:

In [1]:
import urllib.request
from pathlib import Path

def load_gutenberg_text(url):
    print("Cargando texto desde:", url, "..")
    response = urllib.request.urlopen(url)
    raw = response.read().decode('utf-8')

    # Encontrar el inicio y fin del contenido real (eliminar header/footer de Gutenberg)
    start = raw.find("*** START OF THE PROJECT GUTENBERG")
    start = raw.find("\n", start) + 1
    end = raw.find("*** END OF THE PROJECT GUTENBERG")

    return raw[start:end]

# URLs de los libros en Project Gutenberg
urls = [
    "https://www.gutenberg.org/files/1661/1661-0.txt",    # The Adventures of Sherlock Holmes
    "https://www.gutenberg.org/files/108/108-0.txt",      # The Return of Sherlock Holmes
    "https://www.gutenberg.org/files/2097/2097-0.txt",    # The Hound of the Baskervilles
    "https://www.gutenberg.org/files/244/244-0.txt"       # A Study in Scarlet
]

# Carga y guarda el texto
text = " "
for url in urls:
  book_text = load_gutenberg_text(url)
  text += book_text


print("Texto cargado:", len(text), "caracteres")

Cargando texto desde: https://www.gutenberg.org/files/1661/1661-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/108/108-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/2097/2097-0.txt ..
Cargando texto desde: https://www.gutenberg.org/files/244/244-0.txt ..
Texto cargado: 1742150 caracteres


# 2) Preprocesamiento

Tokeniza y limpia el texto:
* Convierte a minúsculas
* Elimina tokens que no son palabras

In [2]:
import spacy
from spacy.lang.en import English

# Inicializar spaCy (solo tokenizador para velocidad)
nlp = English(disable=['tagger','parser','ner'])

def tokenize(text):
    doc = nlp(text)
    return [token.text.lower() for token in doc
            if token.text.strip() and not token.is_punct]

# Dividir en oraciones y tokenizar
corpus_sentences = []
for line in text.split('\n'):
    if line.strip():  # ignorar líneas vacías
        tokens = tokenize(line)
        if tokens:  # ignorar líneas sin tokens válidos
            corpus_sentences.append(tokens)

print("Total de oraciones procesadas:", len(corpus_sentences))

Total de oraciones procesadas: 27683


# 3) Modelo Word2Vec

---



In [3]:
from gensim.models import Word2Vec

##3.1) Entrenamiento

In [4]:
w2v_model = Word2Vec(sentences=corpus_sentences,
                    vector_size=300,      # Dimensión del vector (300)
                    window=8,             # Ventana más amplia para capturar más contexto (8)
                    min_count=2,          # Filtrar palabras poco frecuentes (5)
                    workers=4,
                    sg=1,                 # Usar Skip-gram
                    epochs= 30,           # Más ciclos de entrenamiento
                    negative= 15,         # Tamaño de Muestra Negativa (Negative sampling)
                    alpha= 0.025,         # Learning rate inicial
                    min_alpha= 0.0001     # Learning rate final
                  )

Guardar modelos

In [5]:
w2v_model.save("sherlock_w2v.model")

## 3.2) Análisis de Similitudes

Cargar vectores

In [6]:
w2v_vectors = w2v_model.wv

print("\nEstadísticas del modelo:")
print("Dimensión de los vectores:", w2v_vectors.vector_size)
print("Número de palabras (Word2Vec):", len(w2v_vectors.index_to_key))


Estadísticas del modelo:
Dimensión de los vectores: 300
Número de palabras (Word2Vec): 8416


In [7]:
print("\nPalabras más similares a 'holmes' (Word2Vec):")
print(w2v_vectors.most_similar('holmes'))


Palabras más similares a 'holmes' (Word2Vec):
[('sherlock', 0.5309870839118958), ('demurely', 0.4677028954029083), ('cheerily', 0.43281084299087524), ('gleefully', 0.4195120930671692), ('approvingly', 0.417387992143631), ('involuntarily', 0.41265785694122314), ('motioning', 0.4117877185344696), ('bungler', 0.40859004855155945), ('yawn', 0.40683627128601074), ('triumphantly', 0.40631672739982605)]


In [8]:
print("\nPalabras más similares a 'crime' (Word2Vec):")
print(w2v_vectors.most_similar('crime'))



Palabras más similares a 'crime' (Word2Vec):
[('committed', 0.5050350427627563), ('records', 0.4515470266342163), ('literature', 0.44971901178359985), ('talent', 0.4442700147628784), ('featureless', 0.44302573800086975), ('detect', 0.4392315149307251), ('deliberate', 0.4383099377155304), ('perpetrator', 0.42234182357788086), ('insane', 0.42127665877342224), ('sots', 0.42112964391708374)]


In [9]:
print("\nSimilitud entre 'crime' y 'art':")
print("Word2Vec:", w2v_vectors.similarity('crime', 'art'))


Similitud entre 'crime' y 'art':
Word2Vec: 0.20315917


## 3.3) Palabras fuera de vocabulario

In [10]:
print("\nPrueba con palabra fuera de vocabulario:")
try:
    print("Word2Vec - Similares a 'investigador':")
    print(w2v_vectors.most_similar('investigador'))
except KeyError:
    print("Word2Vec no puede manejar palabras fuera de vocabulario")


Prueba con palabra fuera de vocabulario:
Word2Vec - Similares a 'investigador':
Word2Vec no puede manejar palabras fuera de vocabulario


## 3.4) Analogías

In [16]:
print("\nAnalogías (Word2Vec):")
result = w2v_vectors.most_similar(positive=['watson', 'crime'],
                                negative=['holmes'])
print("watson:holmes como crime:?")
print(result)


Analogías (Word2Vec):
holmes:police como crime:?
[('comparison', 0.3013310730457306), ('contemplation', 0.2806171774864197), ('composition', 0.27662739157676697), ('reliable', 0.2758445143699646), ('unique', 0.2752229869365692), ('annals', 0.27064117789268494), ('inference', 0.27038633823394775), ('capable', 0.26961779594421387), ('exploit', 0.2694958746433258), ('presuming', 0.2685224115848541)]


#4) Modelo FastText


In [17]:
from gensim.models import FastText

##4.1) Entrenamiento

In [18]:
ft_model = FastText(sentences=corpus_sentences,
                   vector_size=300,    # Aumentar dimensionalidad (300)
                   window=8,           # Ventana más amplia para capturar más contexto (8)
                   min_count=2,        # Mantener min_count bajo para capturar más variantes
                   workers=4,
                   sg=1,               # Skip-gram para mejor calidad
                   min_n=2,            # Tamaño mínimo de n-gramas
                   max_n=6,            # Tamaño máximo de n-gramas (aumentado para capturar más patrones)
                   epochs=30,          # Más ciclos de entrenamiento (30)
                   word_ngrams=1,      # Habilitar n-gramas de palabras
                   negative=15,        # Más muestras negativas
                   alpha=0.025,        # Learning rate inicial
                   min_alpha=0.0001    # Learning rate final
              )

Almacenamiento del modelo:

In [19]:
ft_model.save("sherlock_ft.model")

## 4.2) Análisis de Similitudes

In [20]:
ft_vectors = ft_model.wv

print("\nEstadísticas del modelo:")
print("Dimensión de los vectores:", ft_vectors.vector_size)
print("Número de palabras (FastText):", len(ft_vectors.index_to_key))


Estadísticas del modelo:
Dimensión de los vectores: 300
Número de palabras (FastText): 8416


In [21]:
print("\nPalabras más similares a 'holmes' (FastText):")
print(ft_vectors.most_similar('holmes'))


Palabras más similares a 'holmes' (FastText):
[('holes', 0.48983535170555115), ('sherlock', 0.4753073453903198), ('soames', 0.4750997722148895), ('holborn', 0.431216299533844), ('holds', 0.4205917418003082), ('hold', 0.41582241654396057), ('holiday', 0.4053857922554016), ('holy', 0.4050311744213104), ('volumes', 0.3983922302722931), ('wolf', 0.3976997137069702)]


In [25]:
print("\nPalabras más similares a 'crime' (Word2Vec):")
print(ft_vectors.most_similar('crime'))


Palabras más similares a 'crime' (Word2Vec):
[('crimes', 0.7936595678329468), ('crib', 0.632940411567688), ('criticism', 0.6317828297615051), ('criminal', 0.6295538544654846), ('criminals', 0.6158126592636108), ('grime', 0.6145389676094055), ('crimson', 0.5981522798538208), ('prime', 0.5892316699028015), ('cripple', 0.5829601287841797), ('crisis', 0.5707933902740479)]


In [23]:
print("\nSimilitud entre 'crime' y 'art':")
print("FastText:", ft_vectors.similarity('crime', 'art'))


Similitud entre 'crime' y 'art':
FastText: 0.041709002


##4.3) Palabras fuera del vocabulario

In [24]:
print("\nPrueba con palabra fuera de vocabulario:")

print("\nFastText - Similares a 'investigador':")
print(ft_vectors.most_similar('investigador'))  # FastText puede generar vectores para palabras nuevas



Prueba con palabra fuera de vocabulario:

FastText - Similares a 'investigador':
[('investigate', 0.9281193614006042), ('investigated', 0.9096882343292236), ('investigating', 0.9091376066207886), ('investigations', 0.8724058866500854), ('investigation', 0.868501603603363), ('invest', 0.8680830001831055), ('investments', 0.76113361120224), ('testimonial', 0.6011866927146912), ('domestic', 0.5770685076713562), ('zest', 0.5431040525436401)]
